In [1]:
%pylab inline
from numba import jit, vectorize, float32, float64


Populating the interactive namespace from numpy and matplotlib


## Create test data
This is equivalent to the first-dimension-concatenated array that results from $\nu_e$ and $\nu_\mu$ fluxes binned at $400 \;E \times 400 \cos\theta$ bins.

The resulting *input* array is then $2\times400\times400$, and the transform that must (effectively) left-multiply that is $3\times 2\times400\times400$.

This hopefully represents a realistic scenario for performing an accurate oscillation calculation.

In [2]:
np.random.seed(0)
xform = np.array(np.random.random_sample((3, 2, 400, 400)), dtype=np.float64)
inputs = np.array(np.random.random_sample((2, 400, 400)), dtype=np.float64)

xform_fp32 = np.array(xform, dtype=np.float32)
inputs_fp32 = np.array(inputs, dtype=np.float32)

input0 = inputs[0,::]
input1 = inputs[1,::]


## Numpy using einsum

### Float64 math on float64 inputs/transforms

In [3]:
%timeit np.einsum('ij..., j...', xform, inputs, dtype=np.float64, casting='unsafe');

100 loops, best of 3: 10.1 ms per loop


In [4]:
output_einsum = np.einsum('ij..., j...', xform, inputs)
print output_einsum.shape

(400, 400, 3)


check that it's doing what I want it to do

In [5]:
x = xform[:,:,1,10]
x

array([[ 0.01560606,  0.18573089],
       [ 0.10818345,  0.34420619],
       [ 0.74177861,  0.28394148]])

In [6]:
i = inputs[:,1,10]
i

array([ 0.89478297,  0.46917306])

In [7]:
o = np.dot(x, i)
o

array([ 0.10110397,  0.25829298,  0.79694856])

In [8]:
output_einsum[1,10,:]

array([ 0.10110397,  0.25829298,  0.79694856])

In [9]:
np.all(o == output_einsum[1,10,:])

True

### Float32 math on float64 inputs/transforms

In [10]:
%timeit np.einsum('ij..., j...', xform, inputs, dtype=np.float32, casting='unsafe');

100 loops, best of 3: 6 ms per loop


### Float32 math on float32 inputs/transforms

In [11]:
%timeit np.einsum('ij..., j...', xform_fp32, inputs_fp32, dtype=np.float32, casting='no');

100 loops, best of 3: 3.08 ms per loop


## Python looping

In [12]:
def apply_python(inputs, transform):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    output = np.empty((N_k, N_l, 3), np.float64)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

### Float64 operands

In [13]:
%timeit apply_python(inputs, xform);

1 loop, best of 3: 750 ms per loop


In [14]:
output_python = apply_python(inputs, xform)
np.all(output_python == output_einsum)

True

### Float32 operands

In [15]:
%timeit apply_python(inputs_fp32, xform_fp32);

1 loop, best of 3: 810 ms per loop


## Numba

### Float64 math on float64 operands

In [16]:
@jit("float64[:,:,:](float64[:,:,:], float64[:,:,:,:])", nopython=False, nogil=True, cache=True)
def apply_numba(inputs, transform):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    output = np.empty((N_k, N_l, 3), float64)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

In [17]:
%timeit apply_numba(inputs, xform)

100 loops, best of 3: 4.79 ms per loop


In [18]:
output_numba = apply_numba(inputs, xform)
np.all(output_numba == output_einsum)

True

### Float32 math on float32 operands

In [19]:
@jit("float32[:,:,:](float32[:,:,:], float32[:,:,:,:])", nopython=True, nogil=True, cache=True)
def apply_numba_fp32(inputs, transform):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    output = np.empty((N_k, N_l, 3), float32)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

In [20]:
%timeit apply_numba_fp32(inputs_fp32, xform_fp32)

100 loops, best of 3: 2.28 ms per loop


### Parallelize

In [19]:
@jit("float32[:,:,:](float32[:,:,:], float32[:,:,:,:])", nopython=True, nogil=True, cache=True)
def apply_numba_fp32(inputs, transform):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    output = np.empty((N_k, N_l, 3), float32)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

# Attempting to compile to GPU

In [27]:
@jit("float32[:,:,:], float32[:,:,:,:], float32[:,:,:]", target='cuda') #nopython=True, nogil=True, cache=True)
def apply_numba_fp32(inputs, transform, output):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    #output = np.empty((N_k, N_l, 3), float32)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

TypingError: Caused By:
Traceback (most recent call last):
  File "/home/justin/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 243, in run
    res = stage()
  File "/home/justin/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 463, in stage_nopython_frontend
    self.locals)
  File "/home/justin/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 780, in type_inference_stage
    infer.propagate()
  File "/home/justin/anaconda2/lib/python2.7/site-packages/numba/typeinfer.py", line 565, in propagate
    raise errors[0]
TypingError: No conversion from array(float32, 3d, A) to none for '$339.2'

Failed at nopython (nopython frontend)
No conversion from array(float32, 3d, A) to none for '$339.2'

In [25]:
@vectorize("float32, float32, float32",
           nopython=True, target='parallel')
def apply_numba_fp32(inputs, transform, output):
    N_k = inputs.shape[1]
    N_l = inputs.shape[2]
    #output = np.empty((N_k, N_l, 3), float32)
    for k in range(N_k):
        for l in range(N_l):
            output[k,l,0] = (
                transform[0,0,k,l]*inputs[0,k,l] +
                transform[0,1,k,l]*inputs[1,k,l]
            )
            output[k,l,1] = (
                transform[1,0,k,l]*inputs[0,k,l] +
                transform[1,1,k,l]*inputs[1,k,l]
            )
            output[k,l,2] = (
                transform[2,0,k,l]*inputs[0,k,l] +
                transform[2,1,k,l]*inputs[1,k,l]
            )
    return output

UntypedAttributeError: Caused By:
Traceback (most recent call last):
  File "/home/justin/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 243, in run
    res = stage()
  File "/home/justin/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 463, in stage_nopython_frontend
    self.locals)
  File "/home/justin/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 780, in type_inference_stage
    infer.propagate()
  File "/home/justin/anaconda2/lib/python2.7/site-packages/numba/typeinfer.py", line 565, in propagate
    raise errors[0]
UntypedAttributeError: Unknown attribute "shape" of type float32
File "<ipython-input-25-1d5d227fed47>", line 4

Failed at nopython (nopython frontend)
Unknown attribute "shape" of type float32
File "<ipython-input-25-1d5d227fed47>", line 4